Session (Cookie) -> 인증된 사용자 정보 -> 서버 안쪽 데이터
- [Name : Value] ; domain/path/expired ...
- Cookie -> Client Browser (temporal path ; webdb ...)
-> requests.request*(get,post,...) **cookies = {k:v}
    cookie jar object <= cookie_jar_from_dict
    cookie 매번 관리 (set-cookie)
-> Session().request* (get,post,...)
    request vs. session.request ; cookie 관리 주체
-> Browser Cookie => Cookie dict, jar

In [ ]:
from requests import request
from requests.compat import *
from bs4 import BeautifulSoup

headers = {
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}